# Notebook 10: Estudo de Caso Completo - Auditoria de Licitações

## Objetivo
Combinar tudo o que aprendemos para simular uma auditoria real de licitações públicas.

## O Cenário
Temos um dataset de licitações contendo:
1.  **Valor Estimado** vs **Valor Vencedor**
2.  **Número de Participantes** (Concorrência)
3.  **Duração do Processo** (Dias)
4.  **Desconto Ofertado (%)**

**Anomalias Procuradas (Red Flags):**
- Participante único + Desconto zero (Jogo de cartas marcadas).
- Valor vencedor MUITO abaixo do estimado (Mergulho de preços / inexequibilidade).
- Valor vencedor MUITO acima (Superfaturamento, se permitido).
- Processos muito rápidos para valores muito altos.

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD # Empirical Cumulative Distribution Functions (Novo e rápido)

plt.rcParams['figure.figsize'] = (12, 6)

## 1. Construção do Dataset de Licitações
Vamos criar um dataset mais realista, com colunas nomeadas e lógica de negócio.

In [ ]:
np.random.seed(42)
n_samples = 1000

# Dados Normais
valor_estimado = np.random.exponential(scale=100000, size=n_samples) + 10000
desconto_medio = np.random.beta(a=2, b=5, size=n_samples) # Descontos variam entre 0% e ~60%
valor_vencedor = valor_estimado * (1 - desconto_medio)
n_participantes = np.random.poisson(lam=4, size=n_samples) + 1
dias_processo = np.random.lognormal(mean=4, sigma=0.5, size=n_samples).astype(int) + 10

df = pd.DataFrame({
    'Valor_Estimado': valor_estimado,
    'Valor_Vencedor': valor_vencedor,
    'Num_Participantes': n_participantes,
    'Dias_Processo': dias_processo,
    'Desconto_Percent': desconto_medio
})

# Inserindo Fraudes (Anomalias)
# Caso 1: Conluio (1 participante, 0 desconto, valor alto)
idx_fraude1 = np.random.choice(n_samples, 20, replace=False)
df.loc[idx_fraude1, 'Num_Participantes'] = 1
df.loc[idx_fraude1, 'Desconto_Percent'] = 0.001
df.loc[idx_fraude1, 'Valor_Vencedor'] = df.loc[idx_fraude1, 'Valor_Estimado']

# Caso 2: Mergulho (Desconto absurdo de 90%)
idx_fraude2 = np.random.choice(n_samples, 10, replace=False)
df.loc[idx_fraude2, 'Desconto_Percent'] = 0.95
df.loc[idx_fraude2, 'Valor_Vencedor'] = df.loc[idx_fraude2, 'Valor_Estimado'] * 0.05

print("Amostra dos dados:")
display(df.head())

## 2. Engenharia de Features
Às vezes, criar features derivadas ajuda mais o modelo do que os dados brutos.
Já temos 'Desconto_Percent', que é uma feature derivada muito forte.

In [ ]:
# Normalizando ou preparando dados para o PyOD
# Vamos focar em 3 dimensões principais para a detecção
cols_to_use = ['Valor_Vencedor', 'Num_Participantes', 'Desconto_Percent', 'Dias_Processo']
X = df[cols_to_use].values

## 3. Modelagem com ECOD e Isolation Forest
Usaremos o **ECOD** (Unsupervised Outlier Detection Using Empirical Cumulative Distribution Functions), que é muito moderno e sem hiperparâmetros (Plug & Play).

In [ ]:
contamination = 0.03 # Esperamos cerca de 30 fraudes em 1000
clf_ecod = ECOD(contamination=contamination)
clf_ecod.fit(X)

df['Score_Anomalia'] = clf_ecod.decision_scores_
df['Eh_Outlier'] = clf_ecod.labels_

## 4. Análise dos Resultados (Auditoria)

In [ ]:
outliers = df[df['Eh_Outlier'] == 1].sort_values('Score_Anomalia', ascending=False)

print(f"Foram detectadas {len(outliers)} licitações suspeitas.")
print("\n--- Top 10 Red Flags para Auditoria ---")
display(outliers.head(10))

### Visualizando os Casos
Vamos ver graficamente onde esses outliers caíram.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Num_Participantes', y='Desconto_Percent', hue='Eh_Outlier', palette={0:'blue', 1:'red'})
plt.title('Participantes x Desconto (Red = Suspeito)')
plt.show()

**Interpretação:**
- Pontos vermelhos no canto inferior esquerdo: 1 Participante e 0 desconto (Conluio/Monopólio).
- Pontos vermelhos no topo: Descontos excessivos (Mergulho).
- O modelo capturou exatamente as lógicas que inserimos, sem que tivéssemos programado regras 'if/else'.